In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from datetime import datetime
from functools import reduce

## Functions
Here we have defined all the useful functions to accomplish the request

In [ ]:
def orario(lista, time_col):
    list_time = [datetime.strptime(t, '%H:%M:%S').time() for t in lista]
    number_review = []
    for i in range(0, len(list_time), 2):
        number_review.append(len((time_col[(time_col >= list_time[i]) & (time_col <= list_time[i + 1])])))
    xx = []
    for i in range(0, len(list_time), 2):
        xx.append(str(list_time[i].hour))
    #xx = ['6am', '11am', '2pm', '5pm', '8pm', '12am', '3am']
    plt.bar(xx, number_review, color = 'salmon')
    plt.yscale('log')
    plt.yticks([2000000, 2500000, 3000000, 3500000, 4000000])
    plt.title('Number of review for each interval of time')
    plt.xlabel('Intervals')
    plt.ylabel('Number of review')
    plt.show()

In [ ]:
def parsedate(time_as_a_unix_timestamp):
    return pd.to_datetime(time_as_a_unix_timestamp, unit = 's')

In [ ]:
def filtro(data, lingue):
    a = pd.DataFrame(columns = dataset.columns)
    for i in range(len(lingue)):
        a = pd.concat([a, data[data.language == lingue[i]]])
    return a

# Load the dataset
we load our dataset and using the function **parsedate** we have changed the format of our timestamp

In [ ]:
dataset = pd.read_csv('steam_reviews.csv')

In [ ]:
dataset = pd.read_csv('steam_reviews.csv', header='infer',
parse_dates=['timestamp_created',
'timestamp_updated', 'author.last_played'],
date_parser = parsedate)

In [ ]:
dataset.columns

In [ ]:
dataset.shape

# RQ1

###  Exploratory Data Analysis (EDA)

To try to better understand our dataset we have made a bunch of plots and tables in which we have tried to catch some information about these reviews received for the applications in Steam.

#### Application more reviewed: 
To start our analysis we have made a pie chart about applications more reviewed. In particular we have decided to pick the first thirty games more reviewed and understand how the number of rewiews is splitted between them. Indeed the percentage written in the slices of the pie plot is referred not to the total number of reviews but the to the sum of reviews written for these thirty more popular games. The choice of thirty is due to make cleaner the plot and because we are interested only in the more popular games. The most talked-about.

In [ ]:
a = pd.Series(dataset.groupby("app_name").app_id.count().sort_values(ascending=False).head(30))
plt.rcParams['figure.figsize'] = (10, 10)
plt.pie(a,
labels = a.index,
explode = [0.1 for value in range(0, a.index.nunique())],
shadow = True, autopct = '%.1f%%')
plt.title('Application name', fontsize = 20)
plt.axis('off')
plt.show()

#### Correlation matrix:
Then we have tried to make a correlation matrix to understand if there are some variables correlated between them 

In [ ]:
fig, ax = plt.subplots(figsize=(13,13)) 
sns.heatmap(dataset.corr(), cbar=True, annot = True, cmap='BrBG', linewidths=.3,fmt='.1g')

We have noticed that there was not any particular correlation between columns except for the ones related to time played by the player therefore we have decided to see in depth these correlations to have clearer information about them. 

In [ ]:
df = pd.DataFrame(dataset,columns=['author.playtime_forever','author.playtime_last_two_weeks',\
                                   'author.playtime_at_review'])
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()


#### Time and Language:
At this point we want to extract some information about the language of the reviews and time when they were written. We have divided the day in three parts: morning (8am-2pm), afternoon (2pm-10pm) and night (10pm-8am). 
So for each part of the day we have grouped the reviews by language, counted them and picked the ten languages more popular.

In this way in our final barplot for each popular language we have the number of reviews written in each part of the day. We have also made a table to explain better the number obtained. 

In [ ]:
arr_1 = np.array((dataset['timestamp_created'].dt.time.astype('str') >= "08:00:00")& (dataset['timestamp_created'].dt.time.astype('str') <= "13:59:59"))
arr_2 = np.array((dataset['timestamp_created'].dt.time.astype('str') >= "14:00:00")& (dataset['timestamp_created'].dt.time.astype('str') <= "21:59:59"))
arr_3 = np.array((dataset['timestamp_created'].dt.time.astype('str') >= "22:00:00")& (dataset['timestamp_created'].dt.time.astype('str') <= "7:59:59"))




In [ ]:
mattina = pd.Series(dataset[arr_1].groupby("language").language.count().sort_values(ascending=False).head(10))
pomeriggio = pd.Series(dataset[arr_2].groupby("language").language.count().sort_values(ascending=False).head(10))
notte = pd.Series(dataset[arr_3].groupby("language").language.count().sort_values(ascending=False).head(10))

In [ ]:
df = mattina.to_frame(name = "8am-2pm")
df["2pm-10pm"]=pomeriggio
df["10pm-8am"]=notte
df['language'] = df.index


In [ ]:
df.set_index(["language"], drop = True)

In [ ]:
ax = df.plot(x = "language", kind ='bar', stacked = False, figsize =(10,8), alpha=1, rot=0)
ax.set_yscale('log')
ax.set_xlabel('languages')
ax.set_ylabel("number reviews")

In this stacked barplot we can see that the majority of the reviews are written during the afternoon while during the night fewer people usually write on Steam. The language more used as expected is English

#### Viral Comments:
In this table we have wanted to look at the ten reviews which have received more comments because we have thought that it could be interesting look at them to understand which comments are popular on Steam. 

In [ ]:
dataset_7 = dataset.sort_values(by=['comment_count'], ascending = False)
dataset_7 = dataset_7.reset_index()

In [ ]:
dataset_7[["author.steamid", "language", "review", "comment_count"]].head(10)

Unfortunately the majority of them are written not in english!

#### Games more played:
In our dataset there is a column in which is stored the time played by that player to that particular game. So we have decided to explore what are the games more played in terms of hours. We have decided to pick the top 20 games because we have thought that 20 is a good trade-off between a clear plot and a meaningful number of games. 

In [ ]:
#dataset_8 = dataset_8[["author.steamid", "author.playtime_forever","app_name"]]
dataset_8 = pd.Series(dataset.groupby("app_name")["author.playtime_forever"].sum().sort_values(ascending=False))
ore_di_gioco = dataset_8.values
giochi = dataset_8.index

In [ ]:
plt.figure(figsize = ((15, 8)))
sns.barplot(x = ore_di_gioco[:20], 
            y = giochi[:20], orient = 'h')
plt.title('TOP 20 games more played in terms of hours', size = 20)
plt.ylabel('Games', size = 14, style = 'italic')
plt.xlabel('Number of hours', size = 14, style = 'italic')
#plt.xscale('log')
plt.xticks(np.arange(1000000000,60000000000,2000000000)) 
plt.show()

In this barplot we have found some confirms: the games more played are also often the games more reviewed that were appeared in the pie chart.

#### Active players:
To conclude this first analysis we have tried to understand what are the players more useful for Steam: we have selected the ten authors that have written the most number of helpful and funny reviews. 

In [ ]:
dataset_9 = pd.Series(dataset[(dataset.votes_helpful > 0)].groupby("author.steamid").votes_helpful.count().sort_values(ascending=False))

dataset_10 = pd.Series(dataset[(dataset.votes_funny > 0)].groupby("author.steamid").votes_funny.count().sort_values(ascending=False))





In [ ]:
pd.concat([dataset_9[:11], dataset_10[:11]], axis=1).reset_index().fillna(0).sort_values(by=['votes_helpful'],ascending=False).reset_index(drop = True)


It's interesting to see that the authors who have written some funny reviews have also written helpful reviews. 

#### Languages and subplots

In [ ]:
print("The total number of languages used to write reviews is ",'\033[1m' +str(len(dataset["language"].unique())) +'\033[0m')

Making a subplot we have been able to visualize all the present languages in the dataset and counting the number of reviews. The two subplots have different measure in y-scales!

In [ ]:
fig=plt.figure(figsize=(25,18))
ax1=fig.add_subplot(2,1,1)
dataset['language'].value_counts().head(10).plot.bar(figsize = (18, 10),title='Top Languages',xlabel='Language',ylabel='Number of Reviews', ax = ax1,rot=0, logy = True, color = "orange")
ax2=fig.add_subplot(2,1,2)
dataset['language'].value_counts().iloc[-18:].plot.bar(figsize = (18, 10),title='Other Languages',xlabel='Language',ylabel='Number of Reviews', ax = ax2,rot=0, color = "orchid")
fig.tight_layout();

#dataset['language'].value_counts().plot.bar(figsize = (18, 7),title='Top Languages',xlabel='Language',ylabel='Number of Reviews', ax = ax1)

# RQ2

### Number of reviews for each application in descending order.

We have decided to make a barplot in which we have counted the number of reviews for the first 50 applications. We have decided 50 because it have seemed to us a good tradeoff to have a clean representation a pick the more reviewed games

In [ ]:
number_review = dataset.groupby("app_name").review_id.count().sort_values(ascending=False)
number_review[0:51].plot.bar(figsize = (18, 7), title=' Number of review', xlabel='Name of application',
ylabel='Number of review', color = "coral", logy = True)
plt.show()


### Best Weighted Vote Score

Each review has a **Weighted Vote Score** that represents the helpfuness score of that review. To extract the weighted  vote score for each game we have computed the mean between all the vote for each application. In this way we have an idea about what applications have received the most helpfulness reviews. Then we have decided to select only average votes above 0.3 because we have considered it a good threshold for the best votes.   

In [ ]:
medie = pd.DataFrame(dataset.groupby("app_name").weighted_vote_score.mean().sort_values(ascending=False))
medie = medie[medie.values > 0.3]
medie

# *NEW*
### Which applications have the most and the least recommendations

In [ ]:
#Most
# recommended. group_by app_name. count all recommended,
# count True recommended and False recommended in separate cols, and percentage of these.
# taking only the useful cols
new_data = dataset[['app_name', 'recommended']]
# count_rec col counts all recommended respectively False and True of an application
new_data['count_rec'] = new_data.groupby(['app_name', 'recommended'], sort=False)['recommended'].transform('count')


In [ ]:
# first 20
new_data.head(20)

In [ ]:
# all_rec col counts all recommedations, False and True together
new_data['all_rec'] = new_data.groupby("app_name", sort=False)['count_rec'].transform('count')

In [ ]:
# first 20
new_data.head(20)

In [ ]:
# final dataframe which contains only the True recommendations
# this means that we can calculate the most and the least recommended apps
final = new_data[(new_data['recommended']==True)].drop_duplicates()

In [ ]:
# first 20
final.head(20)

In [ ]:
# perc_rec calculates the percentage recommendation
final['perc_rec'] = (final['count_rec']/final['all_rec'])*100
# drop not useful cols
final.drop(['recommended', 'count_rec'], axis=1, inplace=True)

In [ ]:
# most recommended, first 50
final.sort_values(by='perc_rec', ascending=False).head(50)

In [ ]:
# least recommended, first 50
final.sort_values(by='perc_rec', ascending=True).head(50)

### How many of these applications were purchased, and how many were given for free?

In [ ]:
# steam_purchase
# taking only the useful cols
new_data1 = dataset[['app_name', 'steam_purchase']]

In [ ]:
# first 20
new_data1.head(20)

In [ ]:
# same modus operandi of counting recommendation
new_data1['count_pur'] = new_data1.groupby(['app_name', 'steam_purchase'], sort=False)['steam_purchase'].transform('count')

In [ ]:
# first 20
new_data1.head(20)

In [ ]:
# taking only the ones purchased
final1 = new_data1[(new_data1['steam_purchase']==True)].drop_duplicates()

In [ ]:
# first 20
final1.head(20)

In [ ]:
# drop not useful col
final1.drop(['steam_purchase'], axis=1, inplace=True)

In [ ]:
# first 20
final1.head(20)

In [ ]:
# received_for_free
# taking only the useful cols
new_data2 = dataset[['app_name', 'received_for_free']]

In [ ]:
# first 20
new_data2.head(20)

In [ ]:
# same modus operandi
new_data2['count_free'] = new_data2.groupby(['app_name', 'received_for_free'], sort=False)['received_for_free'].transform('count')

In [ ]:
# first 20
new_data2.head(20)

In [ ]:
# take only the ones received_for_free
final2 = new_data2[(new_data2['received_for_free']==True)].drop_duplicates()

In [ ]:
# first 20
final2.head(20)

In [ ]:
# drop not useful col
final2.drop(['received_for_free'], axis=1, inplace=True)

In [ ]:
# first 20
final2.head(20)

In [ ]:
# now it's time to calculate the final result, by doing a merge of the final dataframes
dfs = [final, final1, final2]
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['app_name'],
                                            how='outer'), dfs)

In [ ]:
# sorting the values in descending order
final_df.sort_values(by='perc_rec', ascending=False)

In [ ]:
# taking the first 40 apps that are most recommended and displaying how many times were
# purchased and how many times were received for free
final_df.head(40)

# *OLD*
### Which applications have the most and the least recommendations

Just to start we have just selected the ten applications that have more and less number of positive recommendations.

In [ ]:
#Most
most = pd.DataFrame(dataset[(dataset.recommended == True)].groupby("app_name").recommended.count().sort_values(ascending=False)).rename(columns={'recommended': 'positive rec'})
most.iloc[:10]



In [ ]:
#Least
most.iloc[-10:]

After extracting the absolute values we have decided to compute the relative values of number of positive recommendations. We have done that computing for each application the ratio between positive recommendation and total number of reviews. This kind of operation have seemed more meaningful to us

In [ ]:
total_recom = dataset.groupby('app_name')['recommended'].count().reset_index()

positive_recom = dataset[(dataset.recommended == True)].groupby('app_name')['recommended'].count().reset_index()

positive_recom = positive_recom.rename(columns={'app_name': 'app_name', 'recommended': 'rec_True'})

dataset_12= positive_recom.merge(total_recom, left_on = 'app_name', right_on = 'app_name')

dataset_12['percent_True'] = dataset_12['rec_True'] / dataset_12['recommended']



In [ ]:
dataset_12 = dataset_12.sort_values(by=["percent_True"], ascending = False).reset_index(drop = True)

most_2 = dataset_12.iloc[0:10]

least_2 = dataset_12.iloc[-10:]

most_2

Just to visualize this table we have made a plot in which we can see the ratio between positive recommendation and total recommendations of games and in addition we can have also an idea of their absolute value. Not all games have the same number of recommendations!

In [ ]:
ax = most_2.plot(x = "app_name", y = ["rec_True", "recommended"], kind ='bar', stacked = False, figsize =(10,8), alpha=1, rot=90, color =["bisque","limegreen"])
ax.set_yscale('log')
ax.set_xlabel('languages')
ax.set_ylabel("number reviews")

In [ ]:
least_2

We do the same kind of plot for games that have the smaller ratio between positive recommendation and total recommendation

In [ ]:
ax = least_2.plot(x = "app_name", y = ["rec_True", "recommended"], kind ='bar', stacked = False, figsize =(10,8), alpha=1, rot=90, color = ["lightblue","purple"])
ax.set_yscale('log')
ax.set_xlabel('languages')
ax.set_ylabel("number reviews")

We can see that if we consider the absolute or relative value of positive recommendations we extract different games. When we consider the absolute value there are some games that have an high number of positive recommendations but also not positive ones so they are in the both list.

### How many of these applications were purchased, and how many were given for free?

Among applications that have the major and the minor *relative* number of positive recommendations we have observed for each application the number of copies that are received for free and the one which are not. We have used only the column **received_for_free** because we have thought that if in this column there is a **True** the game is received for free if instead there is **False** the game is purchased.

We have considered games that have more positive recommendations relative to their number of reviews and not considered games that have more positive recommendations in absolute value because we have thought that in this way the dataframe that we will obtain are more meaningful.

In [ ]:
most_1 = list(most_2.app_name)
new_data = dataset[(dataset["app_name"].isin(most_1))]
pd.DataFrame(new_data.groupby(["app_name", "received_for_free"]).received_for_free.count())

In [ ]:
least_1 = list(least_2.app_name)
new_data_2 = dataset[(dataset["app_name"].isin(least_1))]
pd.DataFrame(new_data_2.groupby(["app_name", "received_for_free"]).received_for_free.count())

In [ ]:
#new_data[new_data.received_for_free == True].groupby("app_name").received_for_free.count()

In [ ]:
#Most
#most_1 = list(most.index)
#new_data = new_data[(new_data["app_name"].isin(most_1))]
#pd.DataFrame(new_data.groupby(["app_name", "received_for_free"]).recommended.count())

In [ ]:
#Least
#least_1 = list(least.index)
#new_data1 = new_data1[(new_data1["app_name"].isin(least_1))]
#pd.DataFrame(new_data1.groupby(["app_name", "received_for_free"]).recommended.count())

# RQ 3

### What is the most common time that authors review an application? For example, authors usually write a review at 17:44.

In [ ]:
# first point
# taking only the timestamp_created col
timestamp_col = np.array(dataset["timestamp_created"].dt.time.astype('str'))

In [ ]:
dict_time = {}
for time in timestamp_col:
    # taking only hour and minute
    new_time = time[:5]
    if new_time not in list(dict_time.key()):
        dict_time[new_time] = 1
    else:
        dict_time[new_time] += 1

In [ ]:
# sorting the dictionary in descending order
dict_time_sorted = {k: v for k, v in sorted(dict_time.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
# returning the most common time (without seconds)
next(iter(dict_time_sorted))

### Create a function that receives as a parameter a list of time intervals and returns the plot the number of reviews for each of the intervals.

Using the function **orario** we can extract for a given list of time interval the number of reviews written in each time interval 


### Use the function that you created in the previous literal to plot the number of reviews between the following time intervals:

In [ ]:
intervalli = ['06:00:00', '10:59:59', '11:00:00', '13:59:59', '14:00:00', '16:59:59',
        '17:00:00', '19:59:59', '20:00:00', '23:59:59', '00:00:00', '02:59:59', '03:00:00',
        '05:59:59']
timestamp_col1 = np.array(dataset["timestamp_created"].dt.time)

In [ ]:
orario(intervalli, timestamp_col1)

On the x-axis for each bar is indicated the starting point of the time interval. We have observed that fewer people have written reviews during the night while the majority of people have written their reviews in the first hours of the morning and in the dinner hours

# RQ4

### What are the top 3 languages used to review applications?

In [ ]:
top_languages = pd.DataFrame(dataset.groupby("language").review_id.count().sort_values(ascending=False).head(3))
top_languages

As expected the majority of the reviews are written in english, chinese and russian!

In [ ]:
top_languages = list(top_languages.index)
top_languages

### Create a function that receives as parameters both the name of a data set and a list of languages’ names and returns a data frame filtered only with the reviews written in the provided languages.

There we have used the function **filtro** to accomplish a dataframe where there are only reviews written in the top 3 languages

In [ ]:
dataset_filter = filtro(dataset, top_languages)
dataset_filter

### Use the function created in the previous literal to find what percentage of these reviews (associated with the top 3 languages) were voted as funny?

For this request we have used the new filtered dataset and for each language we have selected the reviews that have received at least one funny vote and then we have computed the ratio between them and all the reviews written in that language.

To compute this percentage we have used **dataset_filter** that is the new dataframe obtained using the previous function **filtro**

In [ ]:
numeratore = []
denominatore = []
rapporto = []
for i in range(len(top_languages)):
    numeratore.append(dataset_filter[(dataset_filter.votes_funny != 0) & (dataset_filter.language == top_languages[i])].votes_funny.count())
    denominatore.append(dataset_filter[dataset_filter.language == top_languages[i]].votes_funny.count())
    rapporto.append(round((numeratore[i]/denominatore[i])*100, 2))
    print("The percentage of reviews written in ",'\033[1m' +top_languages[i]+'\033[0m'," that has received at least a funny vote is "'\033[1m' +str(rapporto[i])+"%"+'\033[0m')
    


At this point we have also wanted to compute the percentage of reviews that have received at least a funny vote among all these three languages. 

In [ ]:
num = dataset_filter[dataset_filter.votes_funny != 0].votes_funny.count()
den = dataset_filter.votes_funny.count()
print("The percentage of reviews written in one of the top 3 language that has received at least a funny vote is ", '\033[1m' +str(round((num/den)*100, 2))+"%"+'\033[0m')



### Use the function created in the literal “a” to find what percentage of these reviews (associated with the top 3 languages) were voted as helpful?

For this request we have used the new filtered dataset and for each language we have selected the reviews that have received at least one helpful vote and then we have computed the ratio between them and all the reviews written in that language.

To compute this percentage we have used **dataset_filter** that is the new dataframe obtained using the previous function **filtro**

In [ ]:
numeratore = []
denominatore = []
rapporto = []
for i in range(len(top_languages)):
    numeratore.append(dataset_filter[(dataset_filter.votes_helpful != 0) & (dataset_filter.language == top_languages[i])].votes_helpful.count())
    denominatore.append(dataset_filter[dataset_filter.language == top_languages[i]].votes_helpful.count())
    rapporto.append(round((numeratore[i]/denominatore[i])*100, 2))
    print("The percentage of reviews written in ",'\033[1m' +top_languages[i]+"%"+'\033[0m'+ " that has received at least a helpful vote is ",'\033[1m' +str(rapporto[i])+"%"+'\033[0m'+"%")



At this point we have also wanted to compute the percentage of reviews that have received at least a helpful vote among all these three languages.

In [ ]:
num = dataset_filter[dataset_filter.votes_helpful != 0].votes_helpful.count()
den = dataset_filter.votes_helpful.count()
print("The percentage of reviews written in one of the top 3 language that has received at least a helpful vote is ", '\033[1m' +str(round((num/den)*100, 2))+"%"+'\033[0m')


# RQ6 


### What is the average time (days and minutes) a user lets pass before he updates a review?

Just to start we have computed the difference between the time when the review is written and time when the review is updated and then we have transformed this difference in terms of days

In [ ]:
dataset['Difference_Days'] = (dataset['timestamp_updated'] - dataset['timestamp_created'])
dataset['Difference_Days'] = dataset['Difference_Days']/np.timedelta64(1,'D')


After that we have deleted who did not update his review because we have thought that is meaningless consider them. Then we have computed the mean between days and the integer part of this number represents the average number of days after an author updates his review. Instead to transform the decimal part in minutes we have to multiply it for 1440 because in one day there are 1440 minutes. We have made a simple proportion: *1 : 1440 = x : (decimal part of our number)*

In [ ]:
dataset_1 = dataset[dataset.Difference_Days != 0]
average = dataset_1.Difference_Days.mean()
minutes = round((average % 1) * 1440, 0)
days = average // 1
print("The average time a user lets pass before he updates a review is ",'\033[1m' +str(days)+'\033[0m' +" days and ",'\033[1m' +str(minutes)+'\033[0m' +" minutes")



On average an author updates his review almost after a year! 

### Plot the top 3 authors that usually update their reviews.

We have used the dataframe **dataset_1** in which there are only the reviews that have been updated. We did not use the starting dataset because we have to extract who are the authors that usually update their reviews so authors that have updated more reviews through time.

In [ ]:
a = pd.Series(dataset_1.groupby('author.steamid').review_id.count().sort_values(ascending=False).head(3))
a

In [ ]:
plt.figure(figsize=(12, 8))
ax = a.plot(kind="bar", color = ["orchid", "orange", "green"], alpha=0.75, rot=0)
ax.set_title("TOP 3 authors that have updated more reviews")
ax.set_xlabel("Steam ID")
ax.set_ylabel("Number of reviews updated")
labels = list(a.values)
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 1, label, ha="center", va="bottom"
    )

We have put the number of reviews over the bars because the second and the third author have updated almost the same number of reviews.

# RQ7


### What’s the probability that a review has a Weighted Vote Score equal to or bigger than 0.5

We have used the definition of probability to compute these values indeed we have count the number of reviews that has a Weighted Vote Score equal to or bigger than 0.5 and this number represents the favourable case (we have stored this number in **casi_fav**)while the number of total case is represented by the number of the lines of our dataset, stored in **casi_tot**. The probability is the ratio between them. 

In [ ]:
casi_fav = dataset[dataset.weighted_vote_score >= 0.5].weighted_vote_score.count()
casi_fav

In [ ]:
casi_tot = dataset.weighted_vote_score.count()

In [ ]:
result_1 = round(casi_fav/casi_tot, 2)
print("The probability is of a review has a Weighted Vote Score equal to or bigger than 0.5 is "+ '\033[1m' +str(result_1)+'\033[0m')

### What’s the probability that a review has at least one vote as funny given that the Weighted Vote Score is bigger than 0.5?

To compute this conditional probability my sample sample will be reduced indeed we have filtered the dataset in such way that we are going to look for reviews with at least one vote as funny just among reviews with Weighted Vote Score is bigger than 0.5.

In [ ]:
dataset_prob = dataset[dataset.weighted_vote_score > 0.5]

In [ ]:
casi_fav_2 = dataset_prob[dataset_prob.votes_funny != 0].votes_funny.count()


Now our sample space in other words the total case are the favourable case used to compute the last probability, **case_fav**: number of reviews that has a Weighted Vote Score equal to or bigger than 0.5

In [ ]:
result_2 = round(casi_fav_2/casi_fav, 2)
print("The conditional probability that a review has at least one vote as funny given that the Weighted Vote Score is bigger than 0.5 is ",'\033[1m' +str(result_2)+'\033[0m')



### Is the probability that “a review has at least one vote as funny” independent of the “probability that a review has a Weighted Vote Score equal or bigger than 0.

To be independent these two events it would happen that the probability of the event: *a review has at least one vote as funny* would be equal to *probability that a review has at least one vote as funny given that the Weighted Vote Score is bigger than 0.5* because in this way the conditioning of the two probability is useless given that they are independent. 

In [ ]:
casi_fav_3 = dataset[dataset.votes_funny != 0].votes_funny.count()


In [ ]:
result_3 = round(casi_fav_3/casi_tot,2)
print("The probability of a review has at least one vote as funny is "+ '\033[1m' +str(result_3)+'\033[0m')

0.12 is different from 0.24 so these two events are **dependent!**

#$TQ_1$
##Question 1
As known, given a random variable $X$, the Quantile function *Q($\cdot$)* with support $\{ p | p \in [0,1] \}$ is the function that computes:
$$
    Q(p)=s \hspace{0.2 cm} |\hspace{0.2 cm} \mathcal{P}(X<=s) = p
$$
Denoting with $A_i$ the i-th element of the vector $A$ of length $n$ and given $k \in [0,n]$, it is possible to see that our algorithm compute:
$$
    alg(A,k)=s \hspace{0.2 cm} |\hspace{0.2 cm} \#\{A_i<=s\} = k
$$
It is then easily possible to perform some trasformations over our algorithm parameters in order to obtain the similarities with the quantile function, i.e.:

1. A shrinkage over our algorithm support space (i.e. $k'=k/n$);
    
2. A shrinkage over our cardinality measure (i.e. $\#\{A_i<=s \}'=\frac{\#\{A_i<=s \}}{n}$);

Substituting into our $alg(A,k)$ it becomes:
\begin{equation}
    alg(A,k')=s\hspace{0.2 cm} |\hspace{0.2 cm} \frac{\#\{A_i<=s\}}{n} = k'
\end{equation}
In a frequentist approach (said $A_r$ a random sample of the vector $A$) we can equal $\frac{\#\{A_i<=s\}}{n}= \mathcal{P}(A_r <= s)$; In words, our algorithm is computing the value $s$ so that the number of elements in the array $A$ smaller or equal to $s$ will be equal to $k$: we can so somehow define our algorithm a "quantile function over a non-normalized support".
##Question 2
Let consider the worst case scenario, i.e. imagine that $k=n$ and that at each iteration the random sample $s$ will always be equal to $A_1$: it basically means that the $s$ satisfying the condition over $k$ will be selected at the $n_{th}-1$ iteration (when the vector $A$ over which we are calling $alg()$ has lenght equal to 2), we can then assume an asymptotical complexity in the worst case scenario (removing costant therms) equal to $\mathcal{O}(n)$.
##Question 3
In the best case scenario, the right $s$ will always be picked up at the first iteration, regardless of $n$=len($A$): the asymptotical complexity will then be equal to $\mathcal{O}(1)$.

#$TQ_2$
##Question 1
Let dive into the interpretation of the given recursive algorithm's complexity. It is clear that, given a particular $n$ and $\forall l$, and expressing with $T(n)$ the time needed to complete the algorithm called with parameter $n$:

\begin{equation}
    T(n) = T\left(\frac{n}{2}\right)\cdot 2 + \left(\frac{n}{2}+1\right)\cdot 3
\end{equation}

Infact, calling **splitSwap(a,l,n)** we will have to solve two times **splitSwap(a,l,n/2)** plus execute 3 operations for each of the $\left(\frac{n}{2}+1\right)$ iterations of the for loop into **swapList(a,l,n)**. Lets compute running times after the expression of $T(n)$:

\begin{equation}
    T\left(\frac{n}{2}\right) = T\left(\frac{n}{2^2}\right)\cdot 2 + \left(\frac{n}{2^2}+1\right)\cdot 3
\end{equation}
\begin{equation}
    T(n) = T\left(\frac{n}{2^2}\right)\cdot 2^2 + \left(\frac{n}{2^2}+1\right)\cdot2 \cdot 3 +\left(\frac{n}{2}+1\right)\cdot 3
\end{equation}
\begin{equation}
    T(n) = T\left(\frac{n}{2^2}\right)\cdot 2^2 + \left(\frac{n}{2}+1\right)\cdot2 \cdot 3 +3
\end{equation}
\begin{equation}
    T\left(\frac{n}{2^2}\right) = T\left(\frac{n}{2^3}\right)\cdot 2 + \left(\frac{n}{2^3}+1\right)\cdot 3
\end{equation}
\begin{equation}
    T(n) = T\left(\frac{n}{2^3}\right)\cdot 2^3 + \left(\frac{n}{2}+1\right)\cdot 3 \cdot 3 +7
\end{equation}
\begin{equation}
    T(n) = T\left(\frac{n}{2^k}\right)\cdot 2^k + \left(\frac{n}{2}+1\right)\cdot k \cdot 3 +log_2(2^k)-1
\end{equation}

Setting $2^k=n \Leftrightarrow k =log_2(n)$ we obtain:

\begin{equation}
    T(n) = T(1)\cdot n + \left(\frac{n}{2}+1\right)\cdot log_2(n) \cdot 3 +log_2(n)-1 \simeq n\cdot log_2(n)
\end{equation}

In the latter we have removed the dependency from factors, constant terms and considered only the term with the biggest growth rate w.r.t $n$. We can than say that the asymptotical complexity of the algorithm is $\mathcal{O}(n\cdot log_2(n))$.

#Question 2
Given an array **a**, an index **l** and a number **n** (considering the scenario where both **len(a)** and **n** are power of 2 numbers), the algorithm output the array **a'** built as follows:

\begin{equation}
    a'[i]=a[i] \hspace{1cm}\forall i \in [0,1,...,l-1]\hspace{1cm}\mbox{if}\hspace{1cm} l \geq 1
\end{equation}

\begin{equation}
     a'[l+i]=a[l+n-i]
\end{equation}

In words, starting from an index **l** of the original array **a**, the algorithm is reversing the position of the first **n** elements of the array. Because of this of course it is required that **l+n** $\leq$ **len(a)**, otherwise the subroutine **swapList()** will raise an error because of the out-of-range index it loops on. Let describe the algorithm's mechanism. Looking at the code, we can assess how the only part of the code actually changing the position of the array's elements is the subroutine **swapList()**. Given a triplet **(a,l,n)**, once **splitSwap()** is called, it will recursively call himself with an **n** halfed call by call (i.e. **n**$^{(1)}$ =**n/2**, **n**$^{(2)}$ =**n**$^{(1)}/2$, **n**$^{(3)}$ =**n**$^{(2)}/2$ and so on). As we can see in the (Fig.1), after $\text{log}_2(n)-1$ steps, the function **splitSwap(a,l,2)** will be called: in its execution both **splitSwap(a,l,1)** and **splitSwap(a,l+1,1)** will **return** (being **n**=1), finally allowing the execution of **swaplist(a,l,2)** (that we will call **final-node-subroutine** $\forall l$) that will exchange the position of the array's elements **a[l]** with **a[l+1]**. Being  **splitSwap(a,l,2)** completed,  **splitSwap(a,l+2,2)** will be called. Similary, at the end of the execution its **final-node-subroutine** will exchange the position of the array's elements **a[l+2]** with **a[l+3]**. Basically the **final-node-subroutines** consider the array (starting from the element $a[l]$) as a sequence of $\frac{n}{2}$ couples of elements and in each couple they exchange the 1st element with the 2nd one.

Recalling that **splitSwap(a,l,2)** and **splitSwap(a,l+2,2)** where called in **splitSwap(a,l,4)**, **swapList(a,l,4)** (that we will call **semi-final-node-subroutine**) will finally be executed, exchanging the position of the array's elements **a[l]** with **a[l+2]** and **a[l+1]** with **a[l+3]**. So the role of **semi-final-node-subroutines** is to consider the array (starting from the element $a[l]$) as a sequence of $\frac{n}{4}$ couples of couples and to exchange the position of the 1st element of the 1st couple with the 1st element of the 2nd couple, and the 2nd element of the 1st couple with the 2nd element of the 2nd couple. Basically, after the execution of all the **final-node-subroutines** and of the  **semi-final-node-subroutines** the position of the 1st group of 4 elements of the original array will be reversed, the same for the 2nd group of 4 elements and so on. We can so climb our recursive function tree from the **final-node-subroutines** up to the top **first-final-node-subroutine** i.e. **swapList(a,l,n)**. We can see the effect of each kind of **subroutine** level over a test array in two examples at (Fig.2,3) recalling that the output of the **first-final-node-subroutine** will be equal to the algorithm's output.

Having assessed that the algorithm complexity is $\simeq O(n\cdot log_2(n))$, it is possible to confirm that the algorithm it's not optimal: infact it is easily possible to write some pseudo-code with a lower complexity than the given algorithm:

$\hspace{1cm}$**def reverse(a,l,n):**

$\hspace{2cm}$**reversed_array=a**

$\hspace{2cm}$**for i in range(n):**

$\hspace{3cm}$**reversed_array[i+l]=a[l+n-i]**

$\hspace{2cm}$**return reversed_array**

We can easily see that the **reverse()** algorithm complexity has now become (removing costant therms and factors) $O(n)$, proving that the **splitSwap()** algorithm was not optimal.

<figure>

  <img src ="https://drive.google.com/uc?export=view&id=1VP65dIPAoMyp2YGBC59AwacUQynJ7rK_" width=40%>
<figcaption align="center"> Fig.1 :Reaching the first final-node-subroutine</figcaption>
</figure>

<figure>

  <img src ="https://drive.google.com/uc?export=view&id=1a5aDf8nmvmhTVc3KpRu3A072Xc2DWfY6" width=140%>
<figcaption align="center"> Fig.2 :Test over a with len(a)=n=16, l=0</figcaption>
</figure>

<figure>

  <img src ="https://drive.google.com/uc?export=view&id=1SH6fTN4Xp8Oiowht0D7VSN5ypnbB_oN6" width=140%>
<figcaption align="center"> Fig.3 :Test over a with len(a)=16, n=8, l=7</figcaption>
</figure>

#$TQ_3$: Knapsack
In this theoretical question we have to face with a NP-complete problem: the Knapsack one. To solve it generally we have to use heuristic solutions but in some cases they fail to provide the optimal solution. 
* The first heuristic solution is a greedy algorithm in which we order the object in increasing order of weight and then visit them sequentially, adding them to the solution as long as the budget is not exceeded. This algorithm does not provide the optimal solution in every situation indeed in my counterexample this greedy algorithm fails: we fix the budget: **W** = 10 and we have three object.


|i    |w_i| v_i|
|-----|---|----|
|1    |4  |3   |
|2    |6  |5   |
|3    |10 |9   |

We have to visit the object sequentially so we are going to pick the first two objects, but we cannot pick the third one because we will exceed the budget. This choice is not optimal because it would be better pick only the third object because its values (9) is greater of the sum of the first two (8).

* In the second heuristic solution we have to order the objects in decreasing order of values, and then visit them sequentially, adding them to the solution if the budget is not exceeded. This algorithm does not provide the optimal solution in each situation indeed in my counterexample this greedy algorithm fails: I have decided to choose the same budget **W** = 10 and the same number of object of the last counterexample. 

|i    |w_i| v_i|
|-----|---|----|
|1    |9  |9   |
|2    |7  |7   |
|3    |3 |3   |

We have to visit the objects sequentially so we are going to pick the first object, but we cannot pick the last two because we will exceed the budget. This choice is not optimal because it would be better pick the second and the third objects because the sum of their values (10) is greater of the first object value (9).

* In the third heuristic solution we have to order them in decreasing relative value ($v_1$/ $w_i$), and then visit them sequentially, adding them to the solution if the budget is not exceeded
This algorithm does not provide the optimal solution in each situation indeed in my counterexample this greedy algorithm fails: I have decided to choose the same budget **W** = 10 and the same number of object of the two last counterexamples. 

|i    |w_i| v_i|
|-----|---|----|
|1    |9  |10   |
|2    |7  |7   |
|3    |3 |3   |

We have to visit the objects sequentially so we are going to pick the first object whose relative value is 1.11 while the one of the other objects is 1. We cannot pick the last two because we will exceed the budget. This choice is not optimal because it would be better pick the second and the third objects because the sum of their values (10) is greater of the first object value (9).